# Codes of "python_multidimensional_list_search"

python_multidimensional_list_search リポジトリの動作確認ができるノートブックです。

詳細は[]()参照。

In [1]:
import collections
import numpy as np

from typing import Union

## ヘルパー関数

計算に必要なもろもろの関数

In [2]:
# リストのネストの深さを求める関数
def nest_depth(item:Union[tuple,list,np.ndarray]):

     if not item:
         return 0
     else:
         if isinstance(item, (tuple,list,np.ndarray)):
             return max([nest_depth(i) for i in item]) + 1
         else:
             return 0


# リストを決められた回数だけ平坦化する関数
def flatten(input_list, flatten_num:int):

     flatten_num = flatten_num - 1

     if flatten_num < 0:
         for item in input_list:
             yield item
     else:
         for item in input_list:
             if isinstance(item, collections.abc.Iterable) and not isinstance(item, (str, bytes)):
                 yield from flatten(item, flatten_num)
             else:
                 yield item


# 不規則な次元を含んだ多次元リストかどうかをチェックする関数
def isirregular(input_list):
     
     flatten_num = nest_depth(input_list) - 2

     flattened_list = list(flatten(input_list, flatten_num))

     return not len(list(set([nest_depth(item) for item in flattened_list]))) == 1

## ヘルパー関数の動作確認

上で定義したヘルパー関数のテスト

In [10]:
# 4種類の配列でテスト
list1 = [[1,2,3],[2,3,4],[3,4,5]]  # 2次元, 不規則次元なし
list2 = [[[11,12,13],[1,15,16],[17,18,1]],[[11,12,13],[11,1,13],[11,12,13]]]  # 3次元, 不規則次元なし
list3 = [1,2,3,[4,5,6]] # 2次元, 不規則次元あり
list4 = [[[1,2,3],[4,5,6],[7,8,9]],[1,2,3],[1,2,3],[11,12,13,[14,15,16]]]  # 3次元, 不規則次元あり

# テストの実行
print(nest_depth(list1),isirregular(list1))  # 2, False ならOK
print(nest_depth(list2),isirregular(list2))  # 3, False ならOK
print(nest_depth(list3),isirregular(list3))  # 2, True ならOK
print(nest_depth(list4),isirregular(list4))  # 3, True ならOK

2 False
3 False
2 True
3 True


## 多次元のリストから任意の要素について検索してインデックスを調べる

調べたい要素はタプルやリストの形で複数渡すことが可能。

複数渡した場合、第3引数について`contain_all = True`とすると、複数要素を**全て含む要素のインデックス**を返す。デフォルトでは一つでも一致する要素のインデックスを返す。

In [25]:
# 多次元のリストから任意の要素について検索してインデックスを調べる関数
def multidimension_search(
                             input_list:Union[tuple,list,np.ndarray],
                             query:Union[int,str,tuple,list,np.ndarray],
                             contains_all=False
                         ):
                        
     if isirregular(input_list):
         print("An irregular dimensional list cannot be saerchable.")
         return
     
     if isinstance(query, (int, str)):
         query = [query]
     
     if nest_depth(input_list) == 1:
         if contains_all:
             return all(map(lambda x:x in input_list, query))
         else:
             return [i for i,l in enumerate(input_list) if any(map(lambda x:x == l, query))]
     elif nest_depth(input_list) > 2:
         return [multidimension_search(sublist,query,contains_all) for sublist in input_list]

     if contains_all:
         return [i for i,l in enumerate(input_list) if all(map(lambda x:x in l, query))]
     else:
         return [i for i,l in enumerate(input_list) if any(map(lambda x:x in l, query))]

## 多次元のリストから任意の要素について検索してインデックスを調べる実行例

list1~4は先のテストと同じ。

In [26]:
# 4種類の配列でテスト
list1 = [[1,2,3],[2,3,4],[3,4,5]]
list2 = [[[11,12,13],[1,15,16],[17,18,1]],[[11,12,13],[11,1,13],[11,12,13]]]
list3 = [1,2,3,[4,5,6]]
list4 = [[[1,2,3],[4,5,6],[7,8,9]],[1,2,3],[1,2,3],[11,12,13,[14,15,16]]]

# [1,2]を含むかどうか調べる
query = [1,2]

In [27]:
# [1,2]のどちらか一方でも入っているか調べる例
print(multidimension_search(list1, query, contains_all=False))  # [0, 1]
print(multidimension_search(list2, query, contains_all=False))  # [[1, 2], [1]]
print(multidimension_search(list3, query, contains_all=False))  # エラーメッセージ + None
print(multidimension_search(list4, query, contains_all=False))  # エラーメッセージ + None

[0, 1]
[[1, 2], [1]]
An irregular dimensional list cannot be saerchable.
None
An irregular dimensional list cannot be saerchable.
None


In [29]:
# [1,2]の全てが入っているか調べる例
print(multidimension_search(list1, query, contains_all=True))  # [0]
print(multidimension_search(list2, query, contains_all=True))  # [[], []]
print(multidimension_search(list3, query, contains_all=True))  # エラーメッセージ + None
print(multidimension_search(list4, query, contains_all=True))  # エラーメッセージ + None

[0]
[[], []]
An irregular dimensional list cannot be saerchable.
None
An irregular dimensional list cannot be saerchable.
None


## 文字列についても確認

In [30]:
# 以下の4種でテスト
string_list1 = [["aaa","bbb","ccc"],["aaa","bbb","ccc"],["aaa","bbb","ccc"]]
string_list2 = [["aaa","aaa","aaa"],["bbb","bbb","bbb"],["ccc","ccc","ccc"]]
string_list3 = [[["aaaa","aaaa","aaaa"],["bbbb","bbbb","bbbb"]],[["bbb","bbb"],["cccc","cccc"]]]
string_list4 = [[["abc","xyz"],["aaa","bbb","ccc"]],[["aaa","xyz"],["abc","abc","abc"]]]

# "aaa","bbb","ccc"について、含まれているインデックスを検索
string_query = ["aaa","bbb","ccc"]

In [31]:
# "aaa","bbb","ccc"のどちらか一方でも入っているか調べる例
print(multidimension_search(string_list1, string_query, contains_all=False))  # [0, 1, 2]
print(multidimension_search(string_list2, string_query, contains_all=False))  # [0, 1, 2]
print(multidimension_search(string_list3, string_query, contains_all=False))  # [[], [0]]
print(multidimension_search(string_list4, string_query, contains_all=False))  # [[1], [0]]

[0, 1, 2]
[0, 1, 2]
[[], [0]]
[[1], [0]]


In [32]:
# "aaa","bbb","ccc"の全てが入っているか調べる例
print(multidimension_search(string_list1, string_query, contains_all=True))  # [0, 1, 2]
print(multidimension_search(string_list2, string_query, contains_all=True))  # []
print(multidimension_search(string_list3, string_query, contains_all=True))  # [[], []]
print(multidimension_search(string_list4, string_query, contains_all=True))  # [[1], []]

[0, 1, 2]
[]
[[], []]
[[1], []]


## 出力結果をboolで確認する拡張関数

`multidimensional_search()`からの返り値を入れると、該当のインデックスが一つでもあれば`True`を、一つも見つかっていなければ`False`を返す拡張関数`item_exist_checker()`。

出力結果をそのまま`bool(search_result)`としてしまうと、`search_result = [[], []]`などの時に`True`が返ってしまうため、存在確認には`item_exist_checker(search_result)`とする必要がある。

個人的にインデックスとして欲しい場面の方が多かったので、boolでの出力は拡張としました。

In [33]:
# 出力結果をboolで確認する拡張関数
def item_exist_checker(search_result):

     if nest_depth(search_result) == 0:
         if search_result == 0:
            return True
         else:
            return bool(search_result)
     else:
         return any([item_exist_checker(res) for res in search_result])

In [36]:
search_result1 = multidimension_search(list1, query, contains_all=True)
search_result2 = multidimension_search(list3, query, contains_all=True)
search_result3 = multidimension_search(string_list1, string_query, contains_all=True)
search_result4 = multidimension_search(string_list3, string_query, contains_all=True)

print(search_result1)
print(search_result2)
print(search_result3)
print(search_result4)

An irregular dimensional list cannot be saerchable.
[0]
None
[0, 1, 2]
[[], []]


In [37]:
print(item_exist_checker(search_result1))  # [0] -> True (item exists)
print(item_exist_checker(search_result2))  # None -> False (item doesn't exists)
print(item_exist_checker(search_result3))  # [0, 1, 2] -> True (item exists)
print(item_exist_checker(search_result4))  # [[], []] -> False (item doesn't exists)

True
False
True
False


## 2次元リストにのみ使えるクイックAPI

多次元リストから要素を検索する`multidimension_search()`は多次元に対応した複雑な作りになっているため、2次元リストから検索したいだけの時にさくっと使えるAPI`two_dim_search()`を実装しておきました。引数は`multidimension_search()`と同じです。

`nest_depth()`や`isirregular()`といった他の関数を読んでいないため、このコードをコピペして貼るだけでどこでも使えます。

依存関係をなくすために`nest_depth()`等による次元判定をしていない（2次元list前提）作りになっているので、他の次元の配列を入れるとエラーが出る点にはご注意ください。

In [38]:
def two_dim_search(
                     input_list:Union[tuple,list,np.ndarray],
                     query:Union[tuple,list,np.ndarray],
                     contains_all=False
                  ):

     if contains_all:
         return [i for i,l in enumerate(input_list) if all(map(lambda x:x in l, query))]
     else:
         return [i for i,l in enumerate(input_list) if any(map(lambda x:x in l, query))]

In [44]:
# 不規則次元を含まない2次元配列のみ入力可能
list1 = [[1,2,3],[2,3,4],[3,4,5]]
string_list1 = [["aaa","bbb","ccc"],["aaa","bbb","ccc"],["aaa","bbb","ccc"]]

# 1,2および"aaa","bbb","ccc"を含むかどうか調べる
query = [1,2]
string_query = ["aaa","bbb","ccc"]

In [45]:
# [1,2]のどちらか一方でも入っているか調べる例
print(two_dim_search(list1, query, contains_all=False))  # [0, 1]

# [1,2]の全てが入っているか調べる例
print(two_dim_search(list1, query, contains_all=True))  # [0]

# "aaa","bbb","ccc"のどちらか一方でも入っているか調べる例
print(two_dim_search(string_list1, string_query, contains_all=False))  # [0, 1, 2]

# "aaa","bbb","ccc"の全てが入っているか調べる例
print(two_dim_search(string_list1, string_query, contains_all=True))  # [0, 1, 2]

[0, 1]
[0]
[0, 1, 2]
[0, 1, 2]
